In [1]:
import torch
import torch_geometric
from torch_geometric.data import Data, DataLoader
import deepchem as dc
from rdkit import Chem
import pandas as pd
import numpy as np
import torch
from torch_geometric.data import Data, InMemoryDataset
import multiprocessing
   
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import SAGEConv , TransformerConv
from torch.optim.lr_scheduler import ReduceLROnPlateau

No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!


Instructions for updating:
experimental_relax_shapes is deprecated, use reduce_retracing instead


Skipped loading modules with transformers dependency. No module named 'transformers'
cannot import name 'HuggingFaceModel' from 'deepchem.models.torch_models' (c:\Users\sarab\miniconda3\envs\ENV4\lib\site-packages\deepchem\models\torch_models\__init__.py)


In [2]:
from torch_geometric.data import InMemoryDataset, Data

class DeepChemToPyGDataset(InMemoryDataset):
    def __init__(self, data_list, transform=None):
        """
        Converts a list of PyG `Data` objects into an InMemoryDataset.
        
        Args:
            data_list (list of Data): List of PyG Data objects.
            transform (callable, optional): Optional transform applied to each data object.
        """
        super(DeepChemToPyGDataset, self).__init__('.', transform, None, None)
        self.data, self.slices = self.collate(data_list)

    def get(self, idx):
        """
        Retrieves the data object at the specified index.
        """
        return super().get(idx)


In [3]:
pyg_dataset = torch.load('../../data/curated dataset/cleaned_frag_pyg_dataset.pth')

C:\Users\sarab\AppData\Local\Temp\ipykernel_26876\3827785572.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  pyg_dataset = torch.load('../../data/curated dataset/cleaned

In [ ]:
node_features = pyg_dataset[0].x.shape[1]  # Should be 134 based on your description
edge_features = pyg_dataset[0].edge_attr.shape[1]  # Should be 6 based on your description


In [ ]:
# Define Transformer Encoder
class TransformerEncoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(TransformerEncoder, self).__init__()
        # Adjust input dimension of the TransformerConv based on the combined feature size
        self.transformer1 = TransformerConv(node_features + edge_features, hidden_dim, heads=4)
        self.transformer2 = TransformerConv(hidden_dim * 4, hidden_dim, heads=4)  # Adjusted input dimension
        self.transformer3 = TransformerConv(hidden_dim * 4, hidden_dim, heads=4)  # Adjusted input dimension
        self.fc_mu = nn.Linear(hidden_dim * 4, latent_dim)  # Output dimension must match hidden_dim * heads

    def forward(self, x, edge_index, edge_attr):
        edge_attr_expanded = torch.zeros((x.size(0), edge_features)).to(x.device)
        edge_attr_expanded[edge_index[0]] = edge_attr
        x = torch.cat([x, edge_attr_expanded], dim=1)

        x = F.relu(self.transformer1(x, edge_index))
        x = F.relu(self.transformer2(x, edge_index))
        x = F.relu(self.transformer3(x, edge_index))
        return self.fc_mu(x), None  # Return mu and None for logvar

# Define Transformer Decoder
class TransformerDecoder(nn.Module):
    def __init__(self, latent_dim, hidden_dim, node_features, edge_features):
        super(TransformerDecoder, self).__init__()
        self.fc = nn.Linear(latent_dim, hidden_dim)
        self.node_decoder = nn.Linear(hidden_dim, node_features)
        self.edge_decoder = nn.Linear(hidden_dim * 2, edge_features)

    def forward(self, z, edge_index):
        h = F.relu(self.fc(z))
        reconstructed_x = self.node_decoder(h)

        # Edge reconstruction
        row, col = edge_index
        edge_h = torch.cat([h[row], h[col]], dim=1)
        reconstructed_edge_attr = self.edge_decoder(edge_h)

        return reconstructed_x, reconstructed_edge_attr

# Define Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, node_features, edge_features, hidden_dim, latent_dim):
        super(Autoencoder, self).__init__()
        self.encoder = TransformerEncoder(node_features, edge_features, hidden_dim, latent_dim)
        self.decoder = TransformerDecoder(latent_dim, hidden_dim, node_features, edge_features)

    def encode(self, x, edge_index, edge_attr):
        return self.encoder(x, edge_index, edge_attr)

    def decode(self, z, edge_index):
        return self.decoder(z, edge_index)

    def forward(self, x, edge_index, edge_attr):
        z, _ = self.encode(x, edge_index, edge_attr)  # Ignore logvar
        reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index)
        return reconstructed_x, reconstructed_edge_attr

    def generate_multiple_outputs(self, x, edge_index, edge_attr, num_samples=5):
        outputs = []
        with torch.no_grad():
            z, _ = self.encode(x, edge_index, edge_attr)
            for _ in range(num_samples):
                # Add noise to z (sampling from normal distribution)
                z_noisy = z + torch.randn_like(z) * 0.5  # Adjust std deviation for noise level
                reconstructed_x, reconstructed_edge_attr = self.decode(z_noisy, edge_index)
                outputs.append((reconstructed_x, reconstructed_edge_attr, edge_index))
        return outputs

    def generate_molecule(self, num_nodes):
        self.eval()
        with torch.no_grad():
            # Sample from the latent space
            z = torch.randn(num_nodes, latent_dim)

            # Create a fully connected edge_index
            edge_index = torch.combinations(torch.arange(num_nodes), 2).t()

            # Decode
            reconstructed_x, reconstructed_edge_attr = self.decode(z, edge_index)

            return reconstructed_x, edge_index, reconstructed_edge_attr


In [ ]:
#Defining the reconstruction loss. It is MSE for GAE
def loss_function(recon_x, x, recon_edge_attr, edge_attr):
    # Reconstruction losses
    node_recon_loss = F.mse_loss(recon_x, x)
    edge_recon_loss = F.mse_loss(recon_edge_attr, edge_attr)

    return node_recon_loss + edge_recon_loss

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Fixed hyperparameter
epochs = 60

In [ ]:
import os
import json
import torch
import optuna
import multiprocessing
import torch.nn.functional as F
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.loader import DataLoader as PyGDataLoader
from torch.utils.data import random_split
import optuna.visualization as vis
import plotly.graph_objects as go
from multiprocessing import cpu_count
def objective(trial):
    # Hyperparameters to tune with specified ranges
    hidden_dim = trial.suggest_int("hidden_dim", 64, 512, step=64)
    latent_dim = trial.suggest_int("latent_dim", 32, 128, step=32)
    learning_rate = trial.suggest_float("learning_rate", 1e-5, 1e-1, log=True)
    batch_size = trial.suggest_categorical("batch_size", [8, 16, 32, 64])
    patience = trial.suggest_int("patience", 3, 10)
    
    # Fixed hyperparameter
    epochs = 60
    print(f"\n[Trial {trial.number}] Starting with parameters: {trial.params}")
    # Split dataset
    train_ratio = 0.8
    num_train = int(len(pyg_dataset) * train_ratio)
    num_valid = len(pyg_dataset) - num_train
    train_dataset, valid_dataset = random_split(pyg_dataset, [num_train, num_valid])
    
    # Create DataLoaders
    train_loader = PyGDataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader = PyGDataLoader(valid_dataset, batch_size=batch_size, shuffle=False)
    
    # Initialize model
    model = Autoencoder(node_features, edge_features, hidden_dim, latent_dim)
    device = torch.device('cpu')
    model = model.to(device)
    
    # Optimizer and Scheduler
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5, verbose=False)
    
    # Training loop
    best_val_loss = float('inf')
    epochs_without_improvement = 0
    
    for epoch in range(epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        
        for batch in train_loader:
            optimizer.zero_grad()
            x = batch.x.to(device)
            edge_index = batch.edge_index.to(device)
            edge_attr = batch.edge_attr.to(device)
            
            # Forward pass
            recon_x, recon_edge_attr = model(x, edge_index, edge_attr)
            
            # Compute loss
            loss = loss_function(recon_x, x, recon_edge_attr, edge_attr)
            loss.backward()
            optimizer.step()
            
            total_train_loss += loss.item()
        
        # Calculate average training loss
        avg_loss = total_train_loss / len(train_loader)
        
        # Validation phase
        model.eval()
        val_loss = 0
        
        with torch.no_grad():
            for val_batch in val_loader:
                x_val = val_batch.x.to(device)
                edge_index_val = val_batch.edge_index.to(device)
                edge_attr_val = val_batch.edge_attr.to(device)
                
                recon_x_val, recon_edge_attr_val = model(x_val, edge_index_val, edge_attr_val)
                val_loss += loss_function(recon_x_val, x_val, recon_edge_attr_val, edge_attr_val).item()
        
        avg_val_loss = val_loss / len(val_loader)
        
        # Print detailed trial information
        print(f"[Trial {trial.number}] Epoch {epoch+1}/{epochs}, Training Loss: {avg_loss:.4f}, Validation Loss: {avg_val_loss:.4f}")
        
        scheduler.step(avg_val_loss)
        
        # Early stopping check
        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            epochs_without_improvement = 0
        else:
            epochs_without_improvement += 1
        
        if epochs_without_improvement >= patience:
            print(f"[Trial {trial.number}] Early stopping after {epoch + 1} epochs.")
            break
    
    return best_val_loss

def run_hyperparameter_tuning():

    num_jobs = min(multiprocessing.cpu_count()-1, 7)  # Use up to 4 cores

    
    
    # Create a study object 
    study = optuna.create_study(direction='minimize')
    
    # Run optimization
    study.optimize(objective, n_trials=200, n_jobs=num_jobs)
    
    # Print best trial details
    best_trial = study.best_trial
    print("\nBest Trial Details:")
    print(f"Best Trial Number: {best_trial.number}")
    print("Best Parameters:", best_trial.params)
    print("Best Value (Objective):", best_trial.value)
    
    # Save best hyperparameters
    with open('best_hyperparameters.json', 'w') as f:
        json.dump(best_trial.params, f, indent=4)
    
    return study, best_trial
study, best_trial = run_hyperparameter_tuning()

# Optional: Visualization (requires plotly)
try:
    import optuna.visualization as vis
    import plotly.graph_objects as go
    
    # Parallel Coordinate Plot
    fig = vis.plot_parallel_coordinate(study)
    fig.show()
    
    # Contour Plot
    fig = vis.plot_contour(study)
    fig.show()
    
    # Optimization History Plot
    fig = vis.plot_optimization_history(study)
    fig.show()
except ImportError:
    print("Plotly or optuna visualization not available. Skipping visualization.")




In [ ]:
from multiprocessing import cpu_count
cpu_count()